<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/text-similarity-works/08_icd_code_highliting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

Reference:

[Rule-based entity recognition](https://spacy.io/usage/rule-based-matching#entityruler)

[PDF-to-TEXT](https://pypi.org/project/pdftotext/)

In [ ]:
!pip -q install spacy
!python -m spacy download en_core_web_sm

!pip install pillow

!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
!pip install -U pdftotext
!pip install PyPDF2
!pip install fitz
!pip install PyMuPDF

In [ ]:
%%shell

sudo apt install tesseract-ocr
sudo apt-get install poppler-utils
pip install tesseract
pip install pytesseract
#pip install Pillow==9.0.0
pip install pdf2image
pip install PyPDF2

Just restart the colab environment.

In [3]:
import pandas as pd
import numpy as np
import re
import time
import os

import pdb

import fitz
import cv2 
import pdftotext
from PyPDF2 import PdfFileReader, PdfFileWriter
from io import BytesIO
from PIL import Image
from matplotlib import pyplot as plt

#import pytesseract
#from pytesseract import Output
#from pdf2image import convert_from_path

import spacy
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English

import nltk

In [ ]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [4]:
!mkdir pdf-files
!mkdir jpg-files
!mkdir txt-files

In [2]:
pdf_files_path = "pdf-files"
jpg_files_path = "jpg-files"
txt_files_path = "txt-files"

In [5]:
#nlp = spacy.load("en_core_web_sm")
nlp = English()

##Define some functions

In [6]:
def split_pdf(pdf_path):
  pdf_in_file = open(pdf_path, "rb")
  pdf = PdfFileReader(pdf_in_file)
  pdf_list = []
  for page in range(pdf.numPages):
      inputpdf = PdfFileReader(pdf_in_file)
      output = PdfFileWriter()
      output.addPage(inputpdf.getPage(page))
      with open(f"{pdf_files_path}/page-{page}.pdf", "wb") as outputStream:
          output.write(outputStream)
          pdf_list.append(f"page-{page}.pdf")
  return pdf_list

In [7]:
def pdf_to_image(pdf_list):
  img_list = []
  i = 0
  for pdf_file in pdf_list:
    pages = convert_from_path(os.path.join(pdf_files_path, pdf_file))
    for page in pages:
      page.save(f"{jpg_files_path}/page-{str(i)}.jpg", "JPEG")
      img_list.append(f"{jpg_files_path}/page-{str(i)}.jpg")
      i = i + 1
  return img_list

In [41]:
def extract_text_from_pdf(pdf_list):
  txt_file_list = []
  i = 0
  for pdf_file in pdf_list:
    with open(os.path.join(pdf_files_path, pdf_file), "rb") as f:
      pdf = pdftotext.PDF(f)
    
    # Read all the text into one string
    pdf_text = "\n\n".join(pdf)

    # write text into file
    with open(f"{txt_files_path}/page-{str(i)}.txt", "a") as f:
      f.write(pdf_text)
    txt_file_list.append(f"{txt_files_path}/page-{str(i)}.txt")
    i += 1
  return txt_file_list

In [ ]:
custom_config = r'--oem 3 --psm 12'

def extract_text_from_image(img_list):
  txt_file_list = []
  for index, image_file in enumerate(img_list):
    image = cv2.imread(image_file)

    # removing noise
    # noiseless_image_bw = cv2.fastNlMeansDenoising(image, None, 40, 7, 21)
    # font thickness
    # image_smoothed = thick_font(noiseless_image_bw) 

    # extract text from image
    txt_data = pytesseract.image_to_string(image, config=custom_config)
    # write text into file
    with open(f"{txt_files_path}/page-{str(index)}.txt", "a") as f:
      f.write(txt_data)
    txt_file_list.append(f"{txt_files_path}/page-{str(index)}.txt")
  return txt_file_list

In [50]:
def get_opt_pattern(icd_10_code):
  # create alternate pattern
  code_arr = icd_10_code.split(".")
  if len(code_arr) > 1:
    code1 = f"{code_arr[0]}. {code_arr[1]}"
    code2 = f"{code_arr[0]} .{code_arr[1]}"
    code3 = f"{code_arr[0]} . {code_arr[1]}"
    return [code1, code2, code3]
  else:
    return icd_10_code

In [53]:
code = get_opt_pattern("G43.109")
code

['G43. 109', 'G43 .109', 'G43 . 109']

In [67]:
def highlight_icd10_code(pdf_page_dict, pdf_file_name):
  pdf_file = fitz.open(pdf_file_name)
  for page_num, page in enumerate(pdf_file):
    if page_num in pdf_page_dict:
      
      for code in pdf_page_dict[page_num]:
        text_to_be_highlighted = code
        highlight = page.search_for(text_to_be_highlighted)
        if len(highlight) == 0:
          alternate_code_list = get_opt_pattern(code)
          for alt_code in alternate_code_list:
            text_to_be_highlighted = alt_code
            highlight = page.search_for(text_to_be_highlighted)
            for inst in highlight:
              highlight = page.add_highlight_annot(inst)
              highlight.update()
              highlight = page.search_for(text_to_be_highlighted)
              print(f"Page-{page_num}: ", code, highlight, end='\n')
              
        for inst in highlight:
          highlight = page.add_highlight_annot(inst)
          highlight.update()
          highlight = page.search_for(text_to_be_highlighted)
          print(f"Page-{page_num}: ", code, highlight, end='\n')
  pdf_file.save(f"{pdf_file_name.split('.')[0]}_output.pdf", garbage=4, deflate=True, clean=True)

In [62]:
def highlight_icd10_code(pdf_page_dict, pdf_file_name):
  pdf_file = fitz.open(pdf_file_name)
  for page_num, page in enumerate(pdf_file):
    if page_num in pdf_page_dict:
      #print(f"Page dict: {pdf_page_dict[page_num]}")
      for code in pdf_page_dict[page_num]:
        text_to_be_highlighted = "G43.109"
        highlight = page.search_for(text_to_be_highlighted)
        if text_to_be_highlighted == "G43.109":
          print(f"highlight: {highlight}")
        for inst in highlight:
          highlight = page.add_highlight_annot(inst)
          highlight.update()
          highlight = page.search_for(text_to_be_highlighted)
          print(f"Page-{page_num}: ", code, highlight, end='\n')
  pdf_file.save(f"{pdf_file_name.split('.')[0]}_output.pdf", garbage=4, deflate=True, clean=True)

In [ ]:
def make_icd_10_code_pattern(icd_10_code_df):
  patterns = []
  for _, row in icd_10_code_df.iterrows():

    # add default pattern
    patterns.append({"label": "ICD-10", "pattern": row["ICD-10"]})

    # create alternate pattern
    code_arr = row["ICD-10"].split(".")
    if len(code_arr) > 1:
      code1 = f"{code_arr[0]}. {code_arr[1]}"
      patterns.append({"label": "ICD-10", "pattern": code1})
  return patterns

In [ ]:
def make_icd_10_code_pattern(icd_10_code_df):
  patterns = []
  for _, row in icd_10_code_df.iterrows():

    # add default pattern
    patterns.append({"label": "ICD-10", "pattern": row["ICD-10"]})

    # create alternate pattern
    code_arr = row["ICD-10"].split(".")
    if len(code_arr) > 1:
      code1 = f"{code_arr[0]}. {code_arr[1]}"
      code2 = f"{code_arr[0]} .{code_arr[1]}"
      code3 = f"{code_arr[0]} . {code_arr[1]}"

    for code_pattern in [code1, code2, code3]:
      patterns.append({"label": "ICD-10", "pattern": code_pattern})
  return patterns

In [13]:
def make_icd_10_code_pattern(icd_10_code_df):
  patterns = []
  for _, row in icd_10_code_df.iterrows():
    patterns.append({"label": "ICD-10", "pattern": row["ICD-10"]})
  return patterns

##Spacy entity rule-matcher

In [ ]:
code = "A00.0"
code_arr = code.split(".")
code_arr

['A00', '0']

In [ ]:
len(code_arr)

2

In [ ]:
code1 = f"{code_arr[0]}. {code_arr[1]}"
code1

'A00. 0'

In [ ]:
code2 = f"{code_arr[0]} .{code_arr[1]}"
code2

'A00 .0'

In [ ]:
code3 = f"{code_arr[0]} . {code_arr[1]}"
code3

'A00 . 0'

In [ ]:
for code in [code1, code2, code3]:
  print(code)

A00. 0
A00 .0
A00 . 0


In [14]:
icd_code_df = pd.read_csv("icd_10_codes.csv")
patterns = make_icd_10_code_pattern(icd_code_df)

ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)
# save to json file
ruler.to_disk("./icd10_code_patterns.jsonl")

In [15]:
nlp.remove_pipe("entity_ruler")

('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x7feda3534050>)

##Text extraction

In [18]:
# Step-1: spliting pdf file
pdf_file_name = "Redacted_Sample_2.pdf"
pdf_list = split_pdf(pdf_file_name)

# Step-2: Extracting text from pdf
txt_list = extract_text_from_pdf(pdf_list)

# Step-3: loading and updating patterns to Spacy
nlp.add_pipe("entity_ruler").from_disk("./icd10_code_patterns.jsonl")

In [42]:
# Step-2: Extracting text from pdf
txt_list = extract_text_from_pdf(pdf_list)

In [40]:
!rm -rf txt-files/*

##ICD-10 code searching

In [37]:
with open(f"{txt_files_path}/page-37.txt", "r") as f:
  one_txt = f.read()
  print(one_txt)
  #doc = nlp(one_txt)
  #print([(ent.text, ent.label_) for ent in doc.ents])

In [38]:
!cat txt-files/page-37.txt

In [ ]:
True if re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", one_txt) else False

True

In [64]:
%%time

pdf_page_dict = {}
# Step-4: Searching ICD-10 code
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    doc = nlp(f.read())
    
    #doc.ents = [e for e in doc.ents if not e.text.isspace()]
    code_list = [ent.text for ent in doc.ents]
    if len(code_list) != 0:
      page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
      pdf_page_dict[page_number] = code_list
      print(f"Page[{txt_file.split('/')[1]}]: {code_list}")
    #print(code_list)

Page[page-2.txt]: ['L14']
Page[page-3.txt]: ['L14', 'L14', 'L22', 'L22']
Page[page-4.txt]: ['L26', 'L26', 'L26', 'L26', 'L26', 'L22', 'L14', 'L22', 'L14']
Page[page-6.txt]: ['L14', 'L14']
Page[page-7.txt]: ['L14', 'L14', 'L14', 'L14']
Page[page-8.txt]: ['L26', 'L26', 'L26']
Page[page-9.txt]: ['L26', 'L26', 'L22', 'L26', 'L22', 'L26', 'L22', 'L22', 'L22']
Page[page-10.txt]: ['L22', 'L26', 'L14']
Page[page-11.txt]: ['L22', 'L22', 'L22']
Page[page-12.txt]: ['L14']
Page[page-18.txt]: ['M54.40']
Page[page-19.txt]: ['G43.109']
Page[page-26.txt]: ['D17.1', 'F43.9']
Page[page-37.txt]: ['G43.109', 'D72.819', 'R00.2', 'R10.30', 'E88.2']
Page[page-39.txt]: ['G43.109', 'D72.819', 'R00.2', 'R10.30']
Page[page-42.txt]: ['G43.109', 'R00.2', 'G43.109', 'D72.819', 'R00.2']
Page[page-74.txt]: ['M54.2', 'R42']
Page[page-82.txt]: ['G43.109', 'R00.2', 'L25.9', 'D72.819']
Page[page-84.txt]: ['G43.109']
Page[page-85.txt]: ['L25.9', 'D72.819']
Page[page-102.txt]: ['G43.109', 'R00.2', 'L25.9', 'Z00.00']
Page[p

In [48]:
%%time

pdf_page_dict = {}
# Step-4: Searching ICD-10 code
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    page_txt = f.read()
    # filter the page that have line number instead of code
    if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
      doc = nlp(page_txt)
      code_list = [ent.text for ent in doc.ents]
      if len(code_list) != 0:
        page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
        pdf_page_dict[page_number] = code_list
        print(f"Page[{txt_file.split('/')[1]}]: {code_list}")

Page[page-18.txt]: ['M54.40']
Page[page-19.txt]: ['G43.109']
Page[page-26.txt]: ['D17.1', 'F43.9']
Page[page-37.txt]: ['G43.109', 'D72.819', 'R00.2', 'R10.30', 'E88.2']
Page[page-39.txt]: ['G43.109', 'D72.819', 'R00.2', 'R10.30']
Page[page-42.txt]: ['G43.109', 'R00.2', 'G43.109', 'D72.819', 'R00.2']
Page[page-74.txt]: ['M54.2', 'R42']
Page[page-82.txt]: ['G43.109', 'R00.2', 'L25.9', 'D72.819']
Page[page-84.txt]: ['G43.109']
Page[page-85.txt]: ['L25.9', 'D72.819']
Page[page-102.txt]: ['G43.109', 'R00.2', 'L25.9', 'Z00.00']
Page[page-105.txt]: ['G43.109', 'R00.2', 'L25.9']
CPU times: user 273 ms, sys: 4.8 ms, total: 278 ms
Wall time: 276 ms


##ICD-10 code highlighting

In [ ]:
0 in pdf_page_dict

False

In [ ]:
pdf_page_dict[2]

['L14', 'L14']

In [ ]:
%%time

# Step-4: Highlighting ICD-10 code into pdf
highlight_icd10_code(pdf_page_dict, pdf_file_name)

In [ ]:
True if re.search("[0-9],L", "3, L14") else False

False

In [ ]:
True if re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", "P 13, L17") else False

True

In [ ]:
pattern = re.compile("(P[ ][0-9]+)(,\s)(L[0-9]+)")

for i, line in enumerate(open("txt-files/pdf-page-3.txt")):
  for match in re.finditer(pattern, line):
    print('Found on line %s: %s' % (i+1, match.group()))

Found on line 2: P 60, L23
Found on line 3: P 61, L14
Found on line 4: P 56, L19
Found on line 7: P 54, L19
Found on line 8: P 49, L5
Found on line 9: P 39, L6
Found on line 10: P 37, L9
Found on line 11: P 35, L14
Found on line 12: P 27, L24
Found on line 13: P 21, L23
Found on line 15: P 11, L20
Found on line 16: P 5, L39
Found on line 18: P 3, L24
Found on line 22: P 77, L38
Found on line 27: P 86, L19
Found on line 29: P 66, L19
Found on line 30: P 29, L19
Found on line 31: P 21, L16
Found on line 32: P 12, L18
Found on line 33: P 3, L22
Found on line 34: P 3, L22
Found on line 38: P 3, L17
Found on line 41: P 79, L31


In [ ]:
# Step-5: Searching ICD-10 code
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    #one_txt = f.read()
    doc = nlp(f.read())
    code_list = [ent.text for ent in doc.ents]
    if len(code_list) != 0:
      print(f"Page[{txt_file.split('/')[1]}]: {code_list}")
    #print([f"Page[{txt_file.split('/')[1]}]: {(ent.text, ent.label_)}" for ent in doc.ents if ent is not None])

Page[pdf-page-2.txt]: ['L14']
Page[pdf-page-3.txt]: ['L14', 'L14', 'L22', 'L22']
Page[pdf-page-4.txt]: ['L26', 'L26', 'L26', 'L26', 'L26', 'L22', 'L14', 'L22', 'L14']
Page[pdf-page-6.txt]: ['L14', 'L14']
Page[pdf-page-7.txt]: ['L14', 'L14', 'L14', 'L14']
Page[pdf-page-8.txt]: ['L26', 'L26', 'L26']
Page[pdf-page-9.txt]: ['L26', 'L26', 'L22', 'L26', 'L22', 'L26', 'L22', 'L22', 'L22']
Page[pdf-page-10.txt]: ['L22', 'L26', 'L14']
Page[pdf-page-11.txt]: ['L22', 'L22', 'L22']
Page[pdf-page-12.txt]: ['L14']
Page[pdf-page-18.txt]: ['M54.40']
Page[pdf-page-19.txt]: ['G43.109']
Page[pdf-page-26.txt]: ['D17.1', 'F43.9']
Page[pdf-page-37.txt]: ['G43.109', 'D72.819', 'R00.2', 'R10.30', 'E88.2']
Page[pdf-page-39.txt]: ['G43.109', 'D72.819', 'R00.2', 'R10.30']
Page[pdf-page-42.txt]: ['G43.109', 'R00.2', 'G43.109', 'D72.819', 'R00.2']
Page[pdf-page-74.txt]: ['M54.2', 'R42']
Page[pdf-page-82.txt]: ['G43.109', 'R00.2', 'L25.9', 'D72.819']
Page[pdf-page-84.txt]: ['G43.109']
Page[pdf-page-85.txt]: ['L25.9

##Tessrect text extraction

In [ ]:
%%time

# Step-1: spliting pdf file
pdf_file_name = "Redacted_Sample_2.pdf"
pdf_list = split_pdf(pdf_file_name)

# Step-2: converting pdf to image
img_list = pdf_to_image(pdf_list)

# Step-4: Extracting text from pdf
txt_list = extract_text_from_image(img_list)

# Step-3: loading and updating patterns to Spacy
nlp.add_pipe("entity_ruler").from_disk("./icd10_code_patterns.jsonl")

In [ ]:
# Step-4: Extracting text from pdf
txt_list = extract_text_from_image(img_list)

# Step-3: loading and updating patterns to Spacy
nlp.add_pipe("entity_ruler").from_disk("./icd10_code_patterns.jsonl")

In [ ]:
%%time

pdf_page_dict = {}
# Step-4: Searching ICD-10 code
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    page_txt = f.read()
    # filter the page that have line number instead of code
    if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
      doc = nlp(page_txt)
      code_list = [ent.text for ent in doc.ents]
      if len(code_list) != 0:
        page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
        pdf_page_dict[page_number] = code_list
        print(f"Page[{txt_file.split('/')[1]}]: {code_list}")

Page[page-17.txt]: ['R64']
Page[page-18.txt]: ['D17.1']
Page[page-19.txt]: ['G43.109']
Page[page-24.txt]: ['M50.30']
Page[page-26.txt]: ['F43.9']
Page[page-37.txt]: ['G43.109', 'D72.819', 'R10.30', 'E88.2']
Page[page-39.txt]: ['G43.109', 'D72.819', 'R10.30']
Page[page-42.txt]: ['G43.109', 'D72.819']
Page[page-74.txt]: ['M50.323', 'M46.92', 'R42']
Page[page-82.txt]: ['G43.109', 'L25.9', 'D72.819']
Page[page-85.txt]: ['R00.2', 'L25.9', 'D72.819']
Page[page-102.txt]: ['G43.109', 'L25.9']
Page[page-105.txt]: ['G43.109', 'L25.9']
CPU times: user 676 ms, sys: 15.5 ms, total: 691 ms
Wall time: 694 ms


In [ ]:
%%time

# Step-4: Highlighting ICD-10 code into pdf
highlight_icd10_code(pdf_page_dict, pdf_file_name)

Page dict: ['R64']
Page dict: ['D17.1']
Page dict: ['G43.109']
Page-19:  G43.109 [Rect(227.2899932861328, 145.4748077392578, 256.4548034667969, 156.68040466308594)]
Page dict: ['M50.30']
Page dict: ['F43.9']
Page-26:  F43.9 [Rect(267.1199951171875, 294.4976806640625, 288.10455322265625, 306.6370849609375)]
Page dict: ['G43.109', 'D72.819', 'R10.30', 'E88.2']
Page-37:  D72.819 [Rect(70.31879425048828, 230.57130432128906, 100.93679809570312, 241.70071411132812)]
Page-37:  R10.30 [Rect(70.31879425048828, 248.33460998535156, 96.43319702148438, 259.4640197753906)]
Page-37:  E88.2 [Rect(70.31879425048828, 257.4552307128906, 91.48410034179688, 268.5846252441406)]
Page dict: ['G43.109', 'D72.819', 'R10.30']
Page-39:  G43.109 [Rect(68.39816284179688, 293.9032287597656, 98.70294952392578, 304.7579650878906)]
Page-39:  D72.819 [Rect(69.1170654296875, 311.899658203125, 98.97944641113281, 322.75439453125)]
Page-39:  R10.30 [Rect(69.1170654296875, 329.643310546875, 94.58699035644531, 340.498046875)]

In [ ]:
with open(f"{txt_files_path}/page-37.txt", "r") as f:
  one_txt = f.read()
  print(one_txt)

, Male,

SF0063198200

Patient Name (005 - Phone Number:

7

4of6

Office/Outpatient Visit

Visit Date: Mon, Jun 24, 2019 9:00 am

Provider: Josephs, Barry, MD (Assistant: Coppage, Jasmine, )

Location: Barry Josephs, M.D. LLG

Electronically signed by Barry Josephs, MD on 06/24/2019 02:46:23 PM

Printed on 06/24/2019 at 3:21 pm.

Assessment:

¥70.0 Preventive Physical Exam (Mild)

434.91 Cerebrovascular accident (Severe)

G43.109 Migraine with aura, not intractable, without status migrainosus

722.91

Cervical disc disorder (Mild)

D72.819 Decreased white blood cell count, unspecified

ROO.2 Palpitations

R10.30 Lower abdominal pain, unspecified

E88.2 Lipomatosis, not elsewhere classified

Plan:

Preventive Physical Exam

PREVENTION & RECOMMENDATIONS:

Overall Health and Lifestyle:

Thank you for coming for the Wellness Program. It was a pleasure to update the history and physical as well as discuss

your laboratory results

Wellness anc Prevention begins with a healthy lifestyle and